In [12]:
#Required libraries for the program

import pandas as pd
import json
import os
import git
import mysql.connector
from sqlalchemy import create_engine



In [13]:
#using this to avoid creating duplicate values inside tables
mydb = mysql.connector.connect(host="localhost",user="root",password="")

mycursor = mydb.cursor(buffered=True)

mycursor.execute('drop database if exists phonepe')

In [14]:
# # #using this to avoid creating duplicate values inside tables
# mydb = mysql.connector.connect(host="localhost",user="root",password="")

# mycursor = mydb.cursor(buffered=True)

# mycursor.execute('drop table if exists phonepe.Agg_transaction')

In [15]:
#created the clone of GIT-HUB repository then,


try:
    repository_url = "https://github.com/PhonePe/pulse.git"

    #destination_directory = "C:/Users/INFO/Desktop/PHONE_PAY/data"

    destination_directory = "Data"

    git.Repo.clone_from(repository_url, destination_directory)
except:
    print("already exists")


already exists


In [16]:
#STATE NAME CORRECTION  
#Agg_state_list--> to get the list of states in India

state_mapping = {
    'andaman-&-nicobar-islands': 'Andaman and Nicobar Islands',
    'tamil-nadu': 'Tamil Nadu',
    'lakshadweep': 'Lakshadweep',
    'telangana': 'Telangana',
    'manipur': 'Manipur',
    'haryana': 'Haryana',
    'gujarat': 'Gujarat',
    'sikkim': 'Sikkim',
    'delhi': 'Delhi',
    'west-bengal': 'West Bengal',
    'uttar-pradesh': 'Uttar Pradesh',
    'goa': 'Goa',
    'punjab': 'Punjab',
    'arunachal-pradesh': 'Arunachal Pradesh',
    'karnataka': 'Karnataka',
    'jammu-&-kashmir': 'Jammu and Kashmir',
    'maharashtra': 'Maharashtra',
    'odisha': 'Odisha',
    'madhya-pradesh': 'Madhya Pradesh',
    'rajasthan': 'Rajasthan',
    'andhra-pradesh': 'Andhra Pradesh',
    'chandigarh': 'Chandigarh',
    'kerala': 'Kerala',
    'chhattisgarh': 'Chhattisgarh',
    'tripura': 'Tripura',
    'mizoram': 'Mizoram',
    'himachal-pradesh': 'Himachal Pradesh',
    'dadra-&-nagar-haveli-&-daman-&-diu': 'Dadra and Nagar Haveli and Daman and Diu',
    'ladakh': 'Ladakh',
    'assam': 'Assam',
    'meghalaya': 'Meghalaya',
    'uttarakhand': 'Uttarakhand',
    'puducherry': 'Puducherry',
    'bihar': 'Bihar',
    'jharkhand': 'Jharkhand',
    'nagaland': 'Nagaland'
}

In [17]:
#Aggregate transaction
#This is to direct the path to get the data as states

path="Data/data/aggregated/transaction/country/india/state/"
Agg_trans_state_list=os.listdir(path)

#This is to extract the data's to create a dataframe

abc={'State':[], 'Year':[],'Quater':[],'Transaction_type':[], 'Transaction_count':[], 'Transaction_amount':[]}

for i in Agg_trans_state_list:
    p_i=path+i+"/"
    if os.path.isdir(p_i):  # Check if it's a directory
        Agg_yr=os.listdir(p_i)
        for j in Agg_yr:
            p_j=p_i+j+"/"
            if os.path.isdir(p_j):  # Check if it's a directory
                Agg_yr_list=os.listdir(p_j)
                for k in Agg_yr_list:
                    p_k=p_j+k
                    if k.endswith('.json'):  # Check if it's a JSON file
                        with open(p_k, 'r') as Data:
                            D=json.load(Data)
                            for z in D['data']['transactionData']:
                                Name=z['name']
                                count=z['paymentInstruments'][0]['count']
                                amount=z['paymentInstruments'][0]['amount']
                                abc['Transaction_type'].append(Name)
                                abc['Transaction_count'].append(count)
                                abc['Transaction_amount'].append(amount)
                                abc['State'].append(i)
                                abc['Year'].append(j)
                                abc['Quater'].append(int(k.strip('.json')))
#Succesfully created a dataframe
Agg_Trans=pd.DataFrame(abc)
Agg_Trans['State'] = Agg_Trans['State'].map(state_mapping)

#connectiong to mysql server
mydb = mysql.connector.connect(host="localhost",user="root",password="")
mycursor = mydb.cursor(buffered=True)

#connecting to mysql alchemy
db_connection_str = f"mysql+mysqlconnector://root@localhost/phonepe"
db_engine = create_engine(db_connection_str)

#creating database and creating table
mycursor.execute('create database if not exists phonepe')
mycursor.execute('USE phonepe')
mycursor.execute('''create table if not exists  Agg_transaction(State VARCHAR(50), 
                                                Year INT(4),
                                                Quater INT(2), 
                                                Transaction_type VARCHAR(40), 
                                                Transaction_count BIGINT, 
                                                Transaction_amount BIGINT )  ''')
mydb.commit()

#migrating to mysql
Agg_Trans.to_sql(name='Agg_transaction', con=db_engine, if_exists='append', index=False)

mycursor.close()
mydb.close()



In [18]:
Agg_Trans

,State,Year,Quater,Transaction_type,Transaction_count,Transaction_amount
0,Andaman and Nicobar Islands,2022,1,Peer-to-peer payments,342649,1.682329e+09
1,Andaman and Nicobar Islands,2022,1,Merchant payments,284298,3.700916e+08
2,Andaman and Nicobar Islands,2022,1,Recharge & bill payments,163401,7.130013e+07
3,Andaman and Nicobar Islands,2022,1,Financial Services,1757,2.869682e+06
4,Andaman and Nicobar Islands,2022,1,Others,2643,2.440322e+06
...,...,...,...,...,...,...
4309,Nagaland,2018,4,Recharge & bill payments,72881,1.661551e+07
4310,Nagaland,2018,4,Peer-to-peer payments,53723,3.589263e+08
4311,Nagaland,2018,4,Merchant payments,10350,1.050063e+07
4312,Nagaland,2018,4,Financial Services,587,2.161274e+05


In [19]:
#Aggregate User
#This is to direct the path to get the data as states

path="Data/data/aggregated/user/country/india/state/"
Agg_user_state_list=os.listdir(path)

#Agg_state_list--> to get the list of states in India

#<------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------>#

#This is to extract the data's to create a dataframe

abc2={'State':[], 'Year':[],'Quater':[],'Registered_PhonePe_users':[], 'PhonePe_app_opens':[]}

for i in Agg_user_state_list:
    p_i=path+i+"/"
    if os.path.isdir(p_i):  # Check if it's a directory
        Agg_yr=os.listdir(p_i)
        for j in Agg_yr:
            p_j=p_i+j+"/"
            if os.path.isdir(p_j):  # Check if it's a directory
                Agg_yr_list=os.listdir(p_j)
                for k in Agg_yr_list:
                    p_k=p_j+k
                    if k.endswith('.json'):  # Check if it's a JSON file
                        with open(p_k, 'r') as Data:
                            D=json.load(Data)
                            for z in D:
                                users=D['data']['aggregated']['registeredUsers']
                                opens=D['data']['aggregated']['appOpens']
                                abc2['Registered_PhonePe_users'].append(users)
                                abc2['PhonePe_app_opens'].append(opens)
                                abc2['State'].append(i)
                                abc2['Year'].append(j)
                                abc2['Quater'].append(int(k.strip('.json')))
#Succesfully created a dataframe
Agg_user=pd.DataFrame(abc2)
Agg_user['State'] = Agg_user['State'].map(state_mapping)

#connectiong to mysql server
mydb = mysql.connector.connect(host="localhost",user="root",password="")
mycursor = mydb.cursor(buffered=True)

#connecting to mysql alchemy
db_connection_str = f"mysql+mysqlconnector://root@localhost/phonepe"
db_engine = create_engine(db_connection_str)

#creating database and creating table
mycursor.execute('USE phonepe')
mycursor.execute('''create table if not exists  Agg_user(State VARCHAR(50), 
                                                Year INT(4),
                                                Quater INT(2), 
                                                Registered_PhonePe_users BIGINT, 
                                                PhonePe_app_opens BIGINT)  ''')
mydb.commit()

#migrating to mysql
Agg_user.to_sql(name='Agg_user', con=db_engine, if_exists='append', index=False)

mycursor.close()
mydb.close()



In [22]:
Agg_user

,State,Year,Quater,Registered_PhonePe_users,PhonePe_app_opens
0,Andaman and Nicobar Islands,2022,1,76288,3847407
1,Andaman and Nicobar Islands,2022,1,76288,3847407
2,Andaman and Nicobar Islands,2022,1,76288,3847407
3,Andaman and Nicobar Islands,2022,1,76288,3847407
4,Andaman and Nicobar Islands,2022,2,80530,5180013
...,...,...,...,...,...
3451,Nagaland,2018,3,32090,0
3452,Nagaland,2018,4,42997,0
3453,Nagaland,2018,4,42997,0
3454,Nagaland,2018,4,42997,0


In [23]:
#Map transaction

#This is to direct the path to get the data as states

path="Data/data/map/transaction/hover/country/india/state/"
Map_trans_state_list=os.listdir(path)

#Agg_state_list--> to get the list of states in India

#<------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------>#

#This is to extract the data's to create a dataframe

abc3={'State':[], 'Year':[],'Quater':[], 'District':[], 'All_Transactions':[], 'Total_payment_value':[]}

for i in Map_trans_state_list:
    p_i=path+i+"/"
    if os.path.isdir(p_i):  # Check if it's a directory
        map_yr=os.listdir(p_i)
        for j in map_yr:
            p_j=p_i+j+"/"
            if os.path.isdir(p_j):  # Check if it's a directory
                map_yr_list=os.listdir(p_j)
                for k in map_yr_list:
                    p_k=p_j+k
                    if k.endswith('.json'):  # Check if it's a JSON file
                        with open(p_k, 'r') as Data:
                            D=json.load(Data)
                            for z in D:
                                dist=D['data']['hoverDataList'][0]['name']
                                totaltrans=D['data']['hoverDataList'][0]['metric'][0]['count']
                                amounts=D['data']['hoverDataList'][0]['metric'][0]['amount']
                                abc3['All_Transactions'].append(totaltrans)
                                abc3['Total_payment_value'].append(amounts)
                                abc3['District'].append(dist)
                                abc3['State'].append(i)
                                abc3['Year'].append(j)
                                abc3['Quater'].append(int(k.strip('.json')))
#Succesfully created a dataframe
map_trans=pd.DataFrame(abc3)
map_trans['State'] = map_trans['State'].map(state_mapping)


#connectiong to mysql server
mydb = mysql.connector.connect(host="localhost",user="root",password="")
mycursor = mydb.cursor(buffered=True)

#connecting to mysql alchemy
db_connection_str = f"mysql+mysqlconnector://root@localhost/phonepe"
db_engine = create_engine(db_connection_str)

#creating database and creating table
mycursor.execute('USE phonepe')
mycursor.execute('''create table if not exists  Map_transaction(State VARCHAR(50), 
                                                Year INT(4),
                                                Quater INT(2), 
                                                District VARCHAR(50),
                                                All_Transactions BIGINT, 
                                                Total_payment_value BIGINT )  ''')
mydb.commit()

#migrating to mysql
map_trans.to_sql(name='Map_transaction', con=db_engine, if_exists='append', index=False)

mycursor.close()
mydb.close()




In [27]:
map_trans

,State,Year,Quater,District,All_Transactions,Total_payment_value
0,Andaman and Nicobar Islands,2022,1,north and middle andaman district,60059,1.616249e+08
1,Andaman and Nicobar Islands,2022,1,north and middle andaman district,60059,1.616249e+08
2,Andaman and Nicobar Islands,2022,1,north and middle andaman district,60059,1.616249e+08
3,Andaman and Nicobar Islands,2022,1,north and middle andaman district,60059,1.616249e+08
4,Andaman and Nicobar Islands,2022,2,north and middle andaman district,72573,1.956932e+08
...,...,...,...,...,...,...
3451,Nagaland,2018,3,mon district,4368,1.068065e+07
3452,Nagaland,2018,4,mon district,7966,2.531512e+07
3453,Nagaland,2018,4,mon district,7966,2.531512e+07
3454,Nagaland,2018,4,mon district,7966,2.531512e+07


In [30]:

#Map user

#This is to direct the path to get the data as states

path="Data/data/map/user/hover/country/india/state/"
Map_user_state_list=os.listdir(path)
abc4={'State':[], 'Year':[],'Quater':[],'District':[],'Total_registered_users':[], 'Total_app_opens':[]}

for i in Map_user_state_list:
    p_i=path+i+"/"
    if os.path.isdir(p_i):  # Check if it's a directory
        map_yr=os.listdir(p_i)
        for j in map_yr:
            p_j=p_i+j+"/"
            if os.path.isdir(p_j):  # Check if it's a directory
                map_yr_list=os.listdir(p_j)
                for k in map_yr_list:
                    p_k=p_j+k
                    if k.endswith('.json'):  # Check if it's a JSON file
                        with open(p_k, 'r') as Data:
                            D=json.load(Data)
                            key_list=list(D['data']['hoverData'].keys())
                            for dist in key_list:
                                abc4['State'].append(i)
                                abc4['Year'].append(j)
                                abc4['Quater'].append(int(k.strip('.json')))
                                abc4['District'].append(dist)
                                abc4['Total_registered_users'].append(D['data']['hoverData'][dist]['registeredUsers'])
                                abc4['Total_app_opens'].append(D['data']['hoverData'][dist]['appOpens'])

map_user=pd.DataFrame(abc4)
map_user['State'] = map_user['State'].map(state_mapping)

chunk_size=1000

#connectiong to mysql server
mydb = mysql.connector.connect(host="localhost",user="root",password="")
mycursor = mydb.cursor(buffered=True)

#connecting to mysql alchemy
db_connection_str = f"mysql+mysqlconnector://root@localhost/phonepe"
db_engine = create_engine(db_connection_str)

#creating database and creating table
mycursor.execute('USE phonepe')
mycursor.execute('''create table if not exists  Map_user(State VARCHAR(50), 
                                                Year INT(4),
                                                Quater INT(2), 
                                                District VARCHAR(100),
                                                Total_registered_users BIGINT, 
                                                Total_app_opens BIGINT )  ''')
mydb.commit()

#migrating to mysql
#map_user.to_sql(name='Map_user', con=db_engine, if_exists='append', index=False)

for i in range(0, len(map_user), chunk_size):
    chunk = map_user[i:i+chunk_size]
    chunk.to_sql(name='Map_user', con=db_engine, if_exists='append', index=False)

mycursor.close()
mydb.close()


In [33]:
map_user

,State,Year,Quater,District,Total_registered_users,Total_app_opens
0,Andaman and Nicobar Islands,2022,1,north and middle andaman district,10720,904869
1,Andaman and Nicobar Islands,2022,1,south andaman district,63487,2368756
2,Andaman and Nicobar Islands,2022,1,nicobars district,2081,573782
3,Andaman and Nicobar Islands,2022,2,north and middle andaman district,11379,963824
4,Andaman and Nicobar Islands,2022,2,south andaman district,66959,3404740
...,...,...,...,...,...,...
17563,Nagaland,2018,4,tuensang district,1261,0
17564,Nagaland,2018,4,peren district,710,0
17565,Nagaland,2018,4,dimapur district,22603,0
17566,Nagaland,2018,4,zunheboto district,1099,0


In [34]:

#top transaction
#This is to direct the path to get the data as states

path="Data/data/top/transaction/country/india/state/"
top_trans_state_list=os.listdir(path)
abc5={'State':[], 'Year':[],'Quater':[], 'District':[],'Districts_total_count':[],'Districts_total_amount':[], 'Postalcodes':[],'Postalcodes_total_count':[],'Postalcodes_total_amount':[]}

for i in top_trans_state_list:
    p_i=path+i+"/"
    if os.path.isdir(p_i):  # Check if it's a directory
        top_yr=os.listdir(p_i)
        for j in top_yr:
            p_j=p_i+j+"/"
            if os.path.isdir(p_j):  # Check if it's a directory
                top_yr_list=os.listdir(p_j)
                for k in top_yr_list:
                    p_k=p_j+k
                    if k.endswith('.json'):  # Check if it's a JSON file
                        with open(p_k, 'r') as Data:
                            D=json.load(Data)
                            if D is not None and 'data' in D and 'states' in D['data'] and 'districts' in D['data'] and 'pincodes' in D['data']:
                                distname = D['data']['districts'][0]['entityName'] if D['data']['districts'] else "Unknown District"
                                postcode = D['data']['pincodes'][0]['entityName'] if D['data']['pincodes'] else "Unknown Postalcode"
                                distcount=D['data']['districts'][0]['metric']['count'] if D['data']['districts'] else "Unknown District count"
                                distamount=D['data']['districts'][0]['metric']['amount'] if D['data']['districts'] else "Unknown District amount"
                                postcodecount=D['data']['pincodes'][0]['metric']['count'] if D['data']['pincodes'] else "Unknown Postalcode count"
                                postcodeamount=D['data']['pincodes'][0]['metric']['amount'] if D['data']['pincodes'] else "Unknown Postalcode amount"
                                abc5['District'].append(distname)
                                abc5['Districts_total_count'].append(distcount)
                                abc5['Districts_total_amount'].append(distamount)
                                abc5['Postalcodes'].append(postcode)
                                abc5['Postalcodes_total_count'].append(postcodecount)
                                abc5['Postalcodes_total_amount'].append(postcodeamount)
                                abc5['State'].append(i)
                                abc5['Year'].append(j)
                                abc5['Quater'].append(int(k.strip('.json')))
#Succesfully created a dataframe
top_trans=pd.DataFrame(abc5)
top_trans['State'] = top_trans['State'].map(state_mapping)


chunk_size=1000

#connectiong to mysql server
mydb = mysql.connector.connect(host="localhost",user="root",password="")
mycursor = mydb.cursor(buffered=True)

#connecting to mysql alchemy
db_connection_str = f"mysql+mysqlconnector://root@localhost/phonepe"
db_engine = create_engine(db_connection_str)

#creating database and creating table
mycursor.execute('USE phonepe')
mycursor.execute('''create table if not exists  Top_transaction(State VARCHAR(50), 
                                                Year INT(4),
                                                Quater INT(2), 
                                                District VARCHAR(100),
                                                Districts_total_count BIGINT, 
                                                Districts_total_amount BIGINT,
                                                Postalcodes INT(6),
                                                Postalcodes_total_count BIGINT,
                                                Postalcodes_total_amount BIGINT
                                                 )  ''')
mydb.commit()

#migrating to mysql

for i in range(0, len(top_trans), chunk_size):
    chunk = top_trans[i:i+chunk_size]
    chunk.to_sql(name='Top_transaction', con=db_engine, if_exists='append', index=False)

mycursor.close()
mydb.close()


In [38]:
top_trans

,State,Year,Quater,District,Districts_total_count,Districts_total_amount,Postalcodes,Postalcodes_total_count,Postalcodes_total_amount
0,Andaman and Nicobar Islands,2022,1,south andaman,705512,1.872991e+09,744101,209941,6.159781e+08
1,Andaman and Nicobar Islands,2022,2,south andaman,954875,2.357327e+09,744101,252722,7.021224e+08
2,Andaman and Nicobar Islands,2022,3,south andaman,1455255,2.967719e+09,744104,497273,9.513703e+08
3,Andaman and Nicobar Islands,2022,4,south andaman,1835592,3.600887e+09,744104,684366,1.309408e+09
4,Andaman and Nicobar Islands,2023,1,south andaman,2030078,3.867385e+09,744103,503053,9.695947e+08
...,...,...,...,...,...,...,...,...,...
859,Nagaland,2020,4,dimapur,422501,1.549414e+09,797112,218369,8.674413e+08
860,Nagaland,2018,1,dimapur,18449,3.833009e+07,797112,8620,1.905946e+07
861,Nagaland,2018,2,dimapur,31758,7.847213e+07,797112,14049,3.158954e+07
862,Nagaland,2018,3,dimapur,44383,1.226767e+08,797112,18409,5.521485e+07


In [39]:

#top user
#This is to direct the path to get the data as states

path="Data/data/top/user/country/india/state/"
top_user_state_list=os.listdir(path)
abc6={'State':[], 'Year':[],'Quater':[],'State_total_registered_user':[], 'District':[],'Districts_registered_user':[], 'Postalcodes':[],'Postalcodes_registered_user':[]}

for i in top_user_state_list:
    p_i=path+i+"/"
    if os.path.isdir(p_i):  # Check if it's a directory
        top_yr=os.listdir(p_i)
        for j in top_yr:
            p_j=p_i+j+"/"
            if os.path.isdir(p_j):  # Check if it's a directory
                top_yr_list=os.listdir(p_j)
                for k in top_yr_list:
                    p_k=p_j+k
                    if k.endswith('.json'):  # Check if it's a JSON file
                        with open(p_k, 'r') as Data:
                            D=json.load(Data)
                            if D is not None and 'data' in D and 'states' in D['data'] and 'districts' in D['data'] and 'pincodes' in D['data']:
                                distname = D['data']['districts'][0]['name'] if D['data']['districts'] else "Unknown District"
                                postcode = D['data']['pincodes'][0]['name'] if D['data']['pincodes'] else "Unknown Postalcode"
                                distuser=D['data']['districts'][0]['registeredUsers'] if D['data']['districts'] else "Unknown District count"
                                postcodeuser=D['data']['pincodes'][0]['registeredUsers'] if D['data']['pincodes'] else "Unknown Postalcode count"
                                stateuser=D['data']['states'][0]['registeredUsers'] if D['data']['states'] else "Unknown Postalcode amount"
                                abc6['State_total_registered_user'].append(distcount)
                                abc6['District'].append(distname)
                                abc6['Districts_registered_user'].append(distamount)
                                abc6['Postalcodes'].append(postcode)
                                abc6['Postalcodes_registered_user'].append(postcodecount)
                                abc6['State'].append(i)
                                abc6['Year'].append(j)
                                abc6['Quater'].append(int(k.strip('.json')))
#Succesfully created a dataframe
top_user=pd.DataFrame(abc6)
top_user['State'] = top_user['State'].map(state_mapping)

chunk_size=1000
#connectiong to mysql server
mydb = mysql.connector.connect(host="localhost",user="root",password="")
mycursor = mydb.cursor(buffered=True)

#connecting to mysql alchemy
db_connection_str = f"mysql+mysqlconnector://root@localhost/phonepe"
db_engine = create_engine(db_connection_str)

#creating database and creating table
mycursor.execute('USE phonepe')
mycursor.execute('''create table if not exists  Top_users(State VARCHAR(50), 
                                                Year INT(4),
                                                Quater INT(2), 
                                                State_total_registered_user BIGINT, 
                                                District VARCHAR(100),
                                                Districts_registered_user BIGINT,
                                                Postalcodes BIGINT,
                                                Postalcodes_registered_user BIGINT
                                                )  ''')
mydb.commit()

#migrating to mysql
for i in range(0, len(top_user), chunk_size):
    chunk = top_user[i:i+chunk_size]
    chunk.to_sql(name='Top_users', con=db_engine, if_exists='append', index=False)

mycursor.close()
mydb.close()

In [43]:
top_user

,State,Year,Quater,State_total_registered_user,District,Districts_registered_user,Postalcodes,Postalcodes_registered_user
0,Andaman and Nicobar Islands,2022,1,68046,south andaman,2.021721e+08,744103,29945
1,Andaman and Nicobar Islands,2022,2,68046,south andaman,2.021721e+08,744103,29945
2,Andaman and Nicobar Islands,2022,3,68046,south andaman,2.021721e+08,744103,29945
3,Andaman and Nicobar Islands,2022,4,68046,south andaman,2.021721e+08,744103,29945
4,Andaman and Nicobar Islands,2023,1,68046,south andaman,2.021721e+08,744103,29945
...,...,...,...,...,...,...,...,...
859,Nagaland,2020,4,68046,dimapur,2.021721e+08,797112,29945
860,Nagaland,2018,1,68046,dimapur,2.021721e+08,797112,29945
861,Nagaland,2018,2,68046,dimapur,2.021721e+08,797112,29945
862,Nagaland,2018,3,68046,dimapur,2.021721e+08,797112,29945


In [44]:
#dataframe creation from sql
#connection to sql
mydb = mysql.connector.connect(host="localhost",user="root",password="",database="phonepe")
mycursor = mydb.cursor(buffered=True)

#agg_transaction_df
mycursor.execute('select * from Agg_transaction')
mydb.commit()
df1=mycursor.fetchall()
Aggregate_transaction = pd.DataFrame(df1,columns=('State', 'Year','Quater','Transaction_type', 'Transaction_count', 'Transaction_amount'))


#agg_user_df
mycursor.execute('select * from Agg_user')
mydb.commit()
df2=mycursor.fetchall()
Aggregate_user = pd.DataFrame(df2,columns=('State', 'Year','Quater','Registered_PhonePe_users', 'PhonePe_app_opens'))

#Map_transaction_df
mycursor.execute('select * from Map_transaction')
mydb.commit()
df3=mycursor.fetchall()
Map_transactions = pd.DataFrame(df3,columns=('State', 'Year','Quater', 'District', 'All_Transactions', 'Total_payment_value'))

#map_user_df
mycursor.execute('select * from Map_user')
mydb.commit()
df4=mycursor.fetchall()
Map_users = pd.DataFrame(df4,columns=('State', 'Year','Quater','District','Total_registered_users', 'Total_app_opens'))

#Top_transaction_df
mycursor.execute('select * from Top_transaction')
mydb.commit()
df5=mycursor.fetchall()
Top_transactions = pd.DataFrame(df5,columns=('State', 'Year','Quater', 'District','Districts_total_count','Districts_total_amount', 'Postalcodes','Postalcodes_total_count','Postalcodes_total_amount'))

#Top_users_df
mycursor.execute('select * from Top_users')
mydb.commit()
df6=mycursor.fetchall()
Top_users_ = pd.DataFrame(df6,columns=('State', 'Year','Quater','State_total_registered_user', 'District','Districts_registered_user', 'Postalcodes','Postalcodes_registered_user'))


In [45]:
Aggregate_transaction
Aggregate_user

Map_transactions
Map_users

Top_transactions
Top_users_

,State,Year,Quater,State_total_registered_user,District,Districts_registered_user,Postalcodes,Postalcodes_registered_user
0,Andaman and Nicobar Islands,2022,1,68046,south andaman,202172095,744103,29945
1,Andaman and Nicobar Islands,2022,2,68046,south andaman,202172095,744103,29945
2,Andaman and Nicobar Islands,2022,3,68046,south andaman,202172095,744103,29945
3,Andaman and Nicobar Islands,2022,4,68046,south andaman,202172095,744103,29945
4,Andaman and Nicobar Islands,2023,1,68046,south andaman,202172095,744103,29945
...,...,...,...,...,...,...,...,...
859,Nagaland,2020,4,68046,dimapur,202172095,797112,29945
860,Nagaland,2018,1,68046,dimapur,202172095,797112,29945
861,Nagaland,2018,2,68046,dimapur,202172095,797112,29945
862,Nagaland,2018,3,68046,dimapur,202172095,797112,29945
